In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import tqdm
import pandas as pd
import gdown
from tqdm import tqdm
import torchvision

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install grad-cam
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image

In [ ]:
!pip install libauc==1.2.0
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import densenet121 as DenseNet121
from libauc.datasets import CheXpert
import libauc

import pandas as pd
import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix

In [ ]:
### Reproducibility
def set_all_seeds(SEED):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 123
set_all_seeds(SEED)

In [ ]:
df_bb = pd.read_csv('/content/gdrive/MyDrive/capstone/BBox_List_2017.csv')
df_data_entry = pd.read_csv('/content/gdrive/MyDrive/capstone/Data_Entry_2017.csv')

In [ ]:
#!unzip /content/gdrive/MyDrive/cs209_final_project/images_small_train_val.zip
!unzip /content/gdrive/MyDrive/cs209_final_project/images_small_test.zip

In [ ]:
df_bb = df_bb[(df_bb['Finding Label'] == 'Atelectasis')|(df_bb['Finding Label'] == 'Cardiomegaly')|(df_bb['Finding Label'] == 'Effusion')]
df_bb = df_bb.reset_index()
n = df_bb.shape[0]
df_bb

,index,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
474,979,00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889,NaN,NaN,NaN
475,980,00025769_001.png,Atelectasis,701.838229,572.491858,103.537778,63.715556,NaN,NaN,NaN
476,981,00016837_002.png,Atelectasis,140.913785,658.962969,271.928889,94.435556,NaN,NaN,NaN
477,982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778,NaN,NaN,NaN


In [ ]:
### Model: Load pre-trained model
model = DenseNet121(pretrained=True, last_activation='sigmoid', activations='relu', num_classes=5) # initialize model

# CPU
#model.load_state_dict(torch.load('/content/gdrive/MyDrive/capstone/trained_auc_model.pt', map_location=torch.device('cpu'))) # load trained model

# GPU
model = model.cuda()
model.load_state_dict(torch.load('/content/gdrive/MyDrive/capstone/trained_auc_model.pt')) # load trained model
model.eval()

In [ ]:
def calculate_IoUs(method, threshold, num = n):
  IoUs = []
  for index in tqdm(range(num)):
    x = torchvision.io.read_image('images_small_test/' + df_bb['Image Index'][index])
    transform_1 = torchvision.transforms.Resize(size = (224, 224))
    transform_2 = torchvision.transforms.ToTensor()
    x = transform_1(x)
    x = transform_2(np.array(x))
    x = x.permute(1, 2, 0)
    x = torch.unsqueeze(x, dim=0)
    x = x.cuda()
    x.shape

    target_layers = [model.features[-1]]

    if method == 'GradCAM':
      cam = GradCAM(model=model, target_layers=target_layers)
    elif method == 'HiResCAM':
      cam = HiResCAM(model=model, target_layers=target_layers)
    elif method == 'GradCAMPlusPlus':
      cam = GradCAMPlusPlus(model=model, target_layers=target_layers)

    grayscale_cam = cam(input_tensor=x)[0, :, :]
    grayscale_cam = (grayscale_cam > threshold).astype(int)

    #rgb_img = x[0, :, :, :]
    #rgb_img = rgb_img.permute(1, 2, 0)
    #rgb_img = rgb_img.cpu().detach().numpy()
    #rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img))
    #visualization = show_cam_on_image(img = rgb_img, mask = grayscale_cam, use_rgb=True)

    #Image.fromarray(visualization, 'RGB')

    # Create figure and axes
    #fig, ax = plt.subplots()

    # Display the image
    #ax.imshow(x[0, 0, :, :])

    # Create a Rectangle patch
    factor = 224/1024
    condition = df_bb['Image Index'] == df_bb['Image Index'][index]
    x_min = df_bb[condition]['Bbox [x'].values[0]*factor
    y_min = df_bb[condition]['y'].values[0]*factor
    w = df_bb[condition]['w'].values[0]*factor
    h = df_bb[condition]['h]'].values[0]*factor
    #rect = patches.Rectangle((x_min, y_min), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    #ax.add_patch(rect)

    #plt.show()

    x_bb = np.zeros((224, 224)).astype(bool)
    x_bb[np.int(y_min):np.int(y_min + h), np.int(x_min):np.int(x_min + w)] = True

    x_sal = (grayscale_cam > threshold).astype(bool)

    x_intersect = np.bitwise_and(x_bb, x_sal)
    x_union = np.bitwise_or(x_bb, x_sal)

    IoU = np.sum(x_intersect)/np.sum(x_union)
    IoUs.append(IoU)

  return IoUs

In [ ]:
saliency_methods = ['GradCAM', 'HiResCAM', 'GradCAMPlusPlus']
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

metrics_df = pd.DataFrame(columns= ['Threshold'] + saliency_methods)

for threshold in thresholds:
  IoUs = []
  for saliency_method in saliency_methods:
    IoU_list = calculate_IoUs(method = saliency_method, threshold = threshold, num = n)
    IoU = np.mean(IoU_list)
    IoUs.append(IoU)
    print(IoU)
  
  metrics_df = pd.concat([metrics_df, pd.DataFrame(data=[[threshold] + IoUs], columns=metrics_df.columns)])

  0%|          | 0/479 [00:00<?, ?it/s]<ipython-input-9-93ed0435418d>:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_bb[np.int(y_min):np.int(y_min + h), np.int(x_min):np.int(x_min + w)] = True
100%|██████████| 479/479 [00:39<00:00, 12.21it/s]


0.13514708111682563


100%|██████████| 479/479 [00:41<00:00, 11.56it/s]


0.13547895873363047


100%|██████████| 479/479 [00:38<00:00, 12.43it/s]


...

0.026224709409235505


100%|██████████| 479/479 [00:36<00:00, 13.26it/s]


0.02366794003610893


100%|██████████| 479/479 [00:38<00:00, 12.51it/s]

0.060608818423013605


In [ ]:
metrics_df

In [ ]:
# Up to index 20
# Works well on: 7, 12
# Works bad on: 2, 3
index = 2
x = torchvision.io.read_image('images_small_test/' + df_bb['Image Index'][index])
transform_1 = torchvision.transforms.Resize(size = (224, 224))
transform_2 = torchvision.transforms.ToTensor()
x = transform_1(x)
x = transform_2(np.array(x))
x = x.permute(1, 2, 0)
x = torch.unsqueeze(x, dim=0)
x = x.cuda()
x.shape

target_layers = [model.features[-1]] # 0 to 11 -> 12 different features

cam = GradCAM(model=model, target_layers=target_layers)

grayscale_cam = cam(input_tensor=x)[0, :, :]
threshold = 0.5
grayscale_cam = (grayscale_cam > threshold).astype(int)

rgb_img = x[0, :, :, :]
rgb_img = rgb_img.permute(1, 2, 0)
rgb_img = rgb_img.cpu().detach().numpy()
rgb_img = (rgb_img - np.min(rgb_img)) / (np.max(rgb_img) - np.min(rgb_img))
visualization = show_cam_on_image(img = rgb_img, mask = grayscale_cam, use_rgb=True)

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
x = x.cpu().detach().numpy()
ax.imshow(x[0, 0, :, :])

ax.imshow(visualization)

# Create a Rectangle patch
factor = 224/1024
condition = df_bb['Image Index'] == df_bb['Image Index'][index]
x_min = df_bb[condition]['Bbox [x'].values[0]*factor
y_min = df_bb[condition]['y'].values[0]*factor
w = df_bb[condition]['w'].values[0]*factor
h = df_bb[condition]['h]'].values[0]*factor
rect = patches.Rectangle((x_min, y_min), w, h, linewidth=1, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
model.features # 12 features, the last one is (norm5)